In [ ]:
#Import packages and set filepaths

In [ ]:
#Data handling nodes, infosource, datasink, etc

In [ ]:
#Unwarping workflow -- nodes associated with it

#Cat's WF
## Unwarping and ME cleaning nodes

# include only the first volume of each PE volume
trim_PEs = Node(Trim(end_index=1),name='trim_PEs')

# merge to 1 file for topup to calculate the fieldcoef
merge_pes = JoinNode(Merge(dimension='t',
                           merged_file='merged_pes.nii.gz'), 
                     name='merge_pes', 
                     joinsource='pes_selectfiles', 
                     joinfield='in_files')

topup = Node(TOPUP(encoding_file=phase_encoding_file), name='topup')

apply_topup = Node(ApplyTOPUP(in_index=[2], encoding_file=phase_encoding_file, 
                              method='jac', out_corrected='func_unwarped.nii.gz'), 
                   name='apply_topup')

In [ ]:
#Unwarping workflow -- actual workflow

#Cat's WF
prepreprocflow = Workflow(name='prepreprocflow')
prepreprocflow.connect([(infosource,pes_selectfiles, [('subjid','subjid')]),
                        (infosource,funcs_selectfiles, [('subjid','subjid')]),
                        (pes_selectfiles, trim_PEs, [('PE_vol','in_file')]), 
                        (trim_PEs, merge_pes, [('out_file','in_files')]), 
                        (merge_pes, topup, [('merged_file','in_file')]),
                        (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                              ('out_movpar','in_topup_movpar')]),
                        (funcs_selectfiles, apply_topup, [('func','in_files')]),
                        (apply_topup, datasink, [('out_corrected','unwarped_funcs')])
                       ])

prepreprocflow.base_dir = workflow_dir
prepreprocflow.write_graph(graph2use='flat')
prepreprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

In [ ]:
#preprocessing workflow -- Nodes

'''Goals:
1. turn off skullstripping in t1_fitter script
2. Register qT1 to DWI b0 volume
3. Double check qt1 and DTI voxel size are both 2mm^3
4. QC DTI -- quantify fd volume by volume --> use FSL motionoutliers
  a. use two thresholds: 1mm, and 1/2 mm
  b. remove volumes that are above each of those thresholds *set iterable on threshold node
  c. Take bvec file and remove volumes that were take out because of motion --> plot remaining directions and b values (color) --> save image
  

In [ ]:
#Preprocessing workflow -- actual workflow